<a href="https://colab.research.google.com/github/David-Delatorre/CECS-456-Project/blob/main/CECS456_ResNet_CNN_Project_V1_David_Delatorre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importing libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import backend as k
!pip install opendatasets
import opendatasets as od

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("andrewmvd/medical-mnist")

# print("Path to dataset files:", path)

# need kaggle username and password
od.download(
	"https://www.kaggle.com/datasets/andrewmvd/medical-mnist")


!pip install -Uqq fastai
from pathlib import Path
from fastai.metrics import error_rate
from fastai.vision.all import *
!pip install timm
import timm
import torch
import pickle
path=Path('../content/medical-mnist');
path

# List files in the dataset to verify
import os
print(os.listdir(path))

Mounted at /content/drive
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: daviddelatorre
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/medical-mnist


100%|██████████| 84.8M/84.8M [00:00<00:00, 154MB/s]



['CXR', 'Hand', 'BreastMRI', 'ChestCT', 'AbdomenCT', 'HeadCT']


In [ ]:
import os
import shutil
from pathlib import Path
import random

# Define the paths to the subdirectories
root_path = path  # Change this to your actual path
categories = ['CXR', 'Hand', 'BreastMRI', 'ChestCT', 'AbdomenCT', 'HeadCT']

# Define paths for training, validation, and test sets
train_path = root_path / 'train'
valid_path = root_path / 'val'
test_path = root_path / 'test'

# Create directories if they don't exist
for split in [train_path, valid_path, test_path]:
    split.mkdir(parents=True, exist_ok=True)

# Create subdirectories within each split for each category
for split in [train_path, valid_path, test_path]:
    for category in categories:
        (split / category).mkdir(parents=True, exist_ok=True)

# Function to split images into train, val, and test sets
def split_data(source_path, train_path, valid_path, test_path, split_ratio=(0.6, 0.2, 0.2)):
    for category in categories:
        # Get all image paths in the category
        category_path = source_path / category
        images = list(category_path.glob('*'))  # All images in the category
        random.shuffle(images)  # Shuffle the images

        # Calculate split sizes
        num_images = len(images)
        train_size = int(num_images * split_ratio[0])
        valid_size = int(num_images * split_ratio[1])

        # Split the images
        train_images = images[:train_size]
        valid_images = images[train_size:train_size+valid_size]
        test_images = images[train_size+valid_size:]

        # Move images to the corresponding directories
        for img in train_images:
            shutil.move(img, train_path / category / img.name)
        for img in valid_images:
            shutil.move(img, valid_path / category / img.name)
        for img in test_images:
            shutil.move(img, test_path / category / img.name)

# Split the data
split_data(root_path, train_path, valid_path, test_path)


test_path = "/content/medical-mnist/test"


train_path="/content/medical-mnist/train"


val_path="/content/medical-mnist/val"


# using data augmentation to prevent overfitting and handle imbalance in dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
Image_gen = ImageDataGenerator(
        rescale = 1/255,
        shear_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.5,2.0],
        width_shift_range = 0.2,
        rotation_range=20,
        fill_mode = 'nearest'
)
val_Datagen = ImageDataGenerator(
        rescale = 1/255
)

train = Image_gen.flow_from_directory(train_path, target_size=(224, 224),
                                       batch_size=32,
                                       class_mode='categorical'
                                       )

validation = Image_gen.flow_from_directory(val_path, target_size=(224, 224),
                                              batch_size=2,
                                              class_mode='categorical'
                                            )

test = val_Datagen.flow_from_directory(test_path, target_size=(224, 224),
                                       batch_size=2,
                                       class_mode='categorical'
                                      )



import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50



# Load the pretrained ResNet model
resnet_base = ResNet50(
    weights='imagenet',  # Use weights pretrained on ImageNet
    include_top=False,   # Exclude the top layer
    input_shape=(224, 224, 3)  # Input shape (match your dataset)
)

# Freeze the base model
resnet_base.trainable = False

# Add custom classification head
model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),  # Pool features from the base model
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dropout(0.5),  # Regularization
    layers.Dense(6, activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train,
    validation_data=validation,
    epochs=30,
    steps_per_epoch=100,
    batch_size=32
)

Found 35372 images belonging to 6 classes.
Found 11790 images belonging to 6 classes.
Found 11792 images belonging to 6 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.2416 - loss: 1.8240 - val_accuracy: 0.4058 - val_loss: 1.4701
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.3797 - loss: 1.4577 - val_accuracy: 0.4830 - val_loss: 1.3160
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.4099 - loss: 1.3829 - val_accuracy: 0.5011 - val_loss: 1.2464
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 249s 3s/step - accuracy: 0.4677 - loss: 1.2993 - val_accuracy: 0.5405 - val_loss: 1.2189
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.4753 - loss: 1.2902 - val_accuracy: 0.5577 - val_loss: 1.1704
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 253s 3s/step - accuracy: 0.4703 - loss: 1.2537 - val_accuracy: 0.5616 - val_loss: 1.1174
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 281s 3s/step - accuracy: 0.5049 - loss: 1.1991 - val_accuracy: 0.5611 - val_loss: 1.0904
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 241s 2s/step - accuracy: 0.5279 - loss: 1.1463 - val_accuracy: 0.586

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.5789 - loss: 1.0080 - val_accuracy: 0.6435 - val_loss: 0.9709
Epoch 13/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 251s 3s/step - accuracy: 0.5432 - loss: 1.1054 - val_accuracy: 0.6417 - val_loss: 0.9691
Epoch 14/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 248s 3s/step - accuracy: 0.5688 - loss: 1.0820 - val_accuracy: 0.6707 - val_loss: 0.9487
Epoch 15/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.5931 - loss: 1.0056 - val_accuracy: 0.6466 - val_loss: 0.9437
Epoch 16/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.5660 - loss: 1.0631